# Titanic survivor prediction


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score
import seaborn as sns

In [14]:
dataset = pd.read_csv(r"D:\prcatice_code\ML_PRACTICE\LOGISTIC_REGRESSION\titanic_survivor_pred\titanic_dataset.csv")

In [15]:
dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,3,"Spector, Mr. Woolf",male,NaN,0.0,0,A.5. 3236,8.05,NaN,S,NaN
1305,1306,1,"Oliva y Ocana, Dona. Fermina",female,39,0.0,0,PC 17758,108.9,C105,C,NaN
1306,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0.0,0,SOTON/O.Q. 3101262,7.25,NaN,S,NaN
1307,1308,3,"Ware, Mr. Frederick",male,NaN,0.0,0,359309,8.05,NaN,S,NaN


In [16]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     1309 non-null   int64  
 2   Pclass       1309 non-null   object 
 3   Name         1309 non-null   object 
 4   Sex          1223 non-null   object 
 5   Age          1132 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   object 
 8   Ticket       1308 non-null   object 
 9   Fare         982 non-null    object 
 10  Cabin        622 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 122.8+ KB


In [17]:
dataset.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex             86
Age            177
SibSp            0
Parch            0
Ticket           1
Fare           327
Cabin          687
Embarked       420
dtype: int64

In [18]:
dataset.isna().mean() * 100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             6.569901
Age            13.521772
SibSp           0.000000
Parch           0.000000
Ticket          0.076394
Fare           24.980901
Cabin          52.482811
Embarked       32.085561
dtype: float64

In [20]:
dataset = dataset.drop("Cabin" , axis = 1)
dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,S
...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,3,"Spector, Mr. Woolf",male,NaN,0.0,0,A.5. 3236,8.05,NaN,NaN
1305,1306,1,"Oliva y Ocana, Dona. Fermina",female,39,0.0,0,PC 17758,108.9,C105,NaN
1306,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0.0,0,SOTON/O.Q. 3101262,7.25,NaN,NaN
1307,1308,3,"Ware, Mr. Frederick",male,NaN,0.0,0,359309,8.05,NaN,NaN


In [24]:
dataset.duplicated("PassengerId").sum()

0